In [1]:
import os 
os.environ['SPARK_HOME']=r'C:/spark/'
os.environ['HADOOP_HOME'] = r'C:/hadoop/'
os.environ['PYSPARK_DRIVER_PYTHON']='jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS']='lab'
os.environ['PYSPARK_PYTHON']='python'

In [3]:
from pyspark.sql import SparkSession
# Crear Sesion spark
spark = SparkSession.builder.appName("App1").getOrCreate()
# Leer la data
customers = spark.read.csv("customers.csv", header=True, inferSchema=True)
agents = spark.read.csv("agents.csv", header=True, inferSchema=True)
calls = spark.read.csv("calls.csv", header=True, inferSchema=True)
# Mostremos la data
customers.show(5)

+----------+----------------+--------------------+--------------------+--------------------+------------+---+
|customerid|            name|          occupation|               email|             company| phonenumber|Age|
+----------+----------------+--------------------+--------------------+--------------------+------------+---+
|         0|    David Melton|          Unemployed|    DMelton@zoho.com|Morris, Winters a...|409-093-0748| 16|
|         1|Michael Gonzalez|             Student|Gonzalez_Michael@...|  Hernandez and Sons|231-845-0673| 19|
|         2|   Amanda Wilson|             Student|Amanda.Wilson75@v...|Mooney, West and ...|844-276-4552| 18|
|         3|   Robert Thomas|Engineer, structural| RThomas@xfinity.com|      Johnson-Gordon|410-404-8000| 25|
|         4|      Eddie Hall|             Surgeon|EddieHall@outlook...|          Dawson LLC|872-287-2196| 30|
+----------+----------------+--------------------+--------------------+--------------------+------------+---+
only showi

In [4]:
spark

In [5]:
# Ejercicio 1
agents.filter((agents.name.like('M%')) | (agents.name.like('%o'))).show(10)

+-------+------------------+
|agentid|              name|
+-------+------------------+
|      0|  Michele Williams|
|      2|Christopher Moreno|
+-------+------------------+



In [7]:
# Ejercicio 2 (Ojo es Case sensitive)
customers \
    .filter(customers.occupation.like('%Engineer%')) \
    .select("occupation") \
    .distinct() \
    .orderBy("occupation").show(10)

+--------------------+
|          occupation|
+--------------------+
|Engineer, aeronau...|
|Engineer, agricul...|
|Engineer, automotive|
|Engineer, biomedical|
|Engineer, broadca...|
|Engineer, buildin...|
|Engineer, civil (...|
|Engineer, civil (...|
|Engineer, communi...|
|Engineer, control...|
+--------------------+
only showing top 10 rows



In [9]:
customers \
    .filter(customers.occupation.like('%Engineer%')) \
    .select("occupation") \
    .distinct() \
    .orderBy("occupation").tail(5)

[Row(occupation='Engineer, production'),
 Row(occupation='Engineer, site'),
 Row(occupation='Engineer, structural'),
 Row(occupation='Engineer, technical sales'),
 Row(occupation='Engineer, water')]

In [12]:
# Ejercicio 3
from pyspark.sql.functions import when, col
customers.select(
    "customerID",
    "name",
    when(col("Age") >= 30, "Yes")
    .when(col("Age") < 30, "No")
    .otherwise("Missing Data")
    .alias("Over30")
).orderBy(col("Name").asc()).show()

+----------+---------------+------+
|customerID|           name|Over30|
+----------+---------------+------+
|       900|Aaron Gutierrez|   Yes|
|       461|  Aaron Hendrix|    No|
|       145| Aaron Mcintyre|    No|
|       622|     Aaron Rose|    No|
|        65|   Adam Jimenez|    No|
|       958|   Adam Leonard|    No|
|       226|      Adam Ward|    No|
|       202| Adrian Aguilar|    No|
|       786|  Alan Chambers|    No|
|       985|  Alan Mitchell|   Yes|
|       881|Alexandra Allen|    No|
|       916| Alexandra Rowe|    No|
|       684|    Alexis Hart|    No|
|       432|  Alexis Riddle|    No|
|       568|      Alice Lee|    No|
|       539|   Alicia Brown|   Yes|
|       830| Alicia Leonard|    No|
|       474|  Alicia Martin|   Yes|
|       918| Alison Vaughan|   Yes|
|       991|Allen Robertson|    No|
+----------+---------------+------+
only showing top 20 rows



In [25]:
# Ejercicio 4
joined_df = customers.alias("Cu") \
    .join(calls.alias("Ca"), col("Ca.customerID") == col("Cu.customerID"), "inner")

#print(joined_df.show())
# Seleccionar columnas de interes y aplicar CASE 
result_df = joined_df.select(
    "Ca.callID",
    col("Cu.customerID"),
    col("Cu.name"),
    col("Cu.occupation"),
    col("Ca.productSold"),
    when(col("Cu.age") >= 30, "Yes")
    .when(col("Cu.age") < 30, "No")
    .otherwise("Missing Data")
    .alias("Over30")
)

result_df.\
    filter(result_df["Cu.occupation"].like("%Engineer%")).\
    orderBy(col("Cu.Name").desc()).drop("occupation").\
    show(5)

+------+----------+-------------+-----------+------+
|callID|customerID|         name|productSold|Over30|
+------+----------+-------------+-----------+------+
|  3688|       699|Willie Greene|          0|   Yes|
|  6270|       699|Willie Greene|          0|   Yes|
|  4150|       699|Willie Greene|          0|   Yes|
|  1817|       699|Willie Greene|          0|   Yes|
|  4900|       699|Willie Greene|          1|   Yes|
+------+----------+-------------+-----------+------+
only showing top 5 rows



In [29]:
# Ejercicio 5
from pyspark.sql.functions import sum, count
from pyspark.sql.functions import col

customers.alias("Cu") \
    .join(calls.alias("Ca"), col("Ca.customerID") == col("Cu.customerID"), "inner").\
    filter(joined_df["Cu.occupation"].like("%Engineer%")).\
    select(
    sum(col("Ca.productSold")).alias("TotalSales"),
    count("*").alias("NCalls")).show(5)

+----------+------+
|TotalSales|NCalls|
+----------+------+
|       502|  2483|
+----------+------+



In [30]:
customers.alias("Cu") \
    .join(calls.alias("Ca"), col("Ca.customerID") == col("Cu.customerID"), "inner").\
    select(
    sum(col("Ca.productSold")).alias("TotalSales"),
    count("*").alias("NCalls")).show(5)

+----------+------+
|TotalSales|NCalls|
+----------+------+
|      2084|  9925|
+----------+------+



In [35]:
# Ejercicio 6
from pyspark.sql.functions import col, count, min, max, avg, sum

calls.alias("C") \
    .join(agents.alias("A"), col("C.agentID") == col("A.agentID"), "inner").\
    filter(col("C.pickeDup") == 1).\
    groupby("A.name").agg(
    count("*").alias("nCalls"),
    min(col("C.duration")).alias("Shortest"),
    max(col("C.duration")).alias("Longest"),
    avg(col("C.duration")).alias("AvgDuration"),
    sum(col("C.productSold")).alias("TotalSales")
    ).orderBy(col("A.name").asc()).show()

+------------------+------+--------+-------+------------------+----------+
|              name|nCalls|Shortest|Longest|       AvgDuration|TotalSales|
+------------------+------+--------+-------+------------------+----------+
|           Agent X|   640|      22|    334|           180.975|       194|
|      Angel Briggs|   591|      12|    362|181.08121827411168|       157|
|Christopher Moreno|   649|      47|    363|  177.979969183359|       189|
|        Dana Hardy|   554|      49|    356|177.20397111913357|       182|
|      Gloria Singh|   662|      36|    349|182.17522658610272|       209|
|    Jocelyn Parker|   621|      40|    336|180.32689210950082|       184|
|      Lisa Cordova|   639|      46|    344|179.21439749608763|       201|
|  Michele Williams|   685|      22|    306|177.88029197080292|       198|
|        Paul Nunez|   648|      -5|    323|  181.070987654321|       194|
|       Randy Moore|   600|      16|    326|           178.595|       177|
|       Todd Morrow|   63

In [37]:
# Ejercicio 7
calls.alias("c") \
    .join(agents.alias("a"), col("c.agentID") == col("a.agentID"), "inner").\
    groupby("a.Name").agg(
    (sum(when(col("c.productSold") == 0, col("c.duration")))/sum(when(col("c.productSold") == 0, 1))).alias("avgWhenNotSold"),
    (sum(when(col("c.productSold") == 1, col("c.duration")))/sum(when(col("c.productSold") == 1, 1))).alias("avgWhenSold")
).orderBy(col("a.name")).show(10)

+------------------+------------------+------------------+
|              Name|    avgWhenNotSold|       avgWhenSold|
+------------------+------------------+------------------+
|           Agent X|109.81705639614856|             185.5|
|      Angel Briggs|108.66160220994475|180.56050955414014|
|Christopher Moreno| 112.4882108183079|182.03703703703704|
|        Dana Hardy| 97.73082706766917|182.30769230769232|
|      Gloria Singh|115.41143654114366|181.10047846889952|
|    Jocelyn Parker|118.99848484848485| 181.7608695652174|
|      Lisa Cordova|110.09470752089136|176.46766169154228|
|  Michele Williams|111.77763496143959|176.18686868686868|
|        Paul Nunez|113.24380165289256| 181.0257731958763|
|       Randy Moore|107.28611898016997|177.47457627118644|
+------------------+------------------+------------------+
only showing top 10 rows

